## 저장된 벡터 db 다운 링크 : https://drive.google.com/drive/folders/1SJYDdkSrHSKsy4ZoMTt-3-8tweAR6_A1?usp=sharing

In [4]:
import os
import glob
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

# BGE-M3 모델 로드 (BAAI/bge-m3 사용)
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 지정된 폴더 경로 설정
folder_path = './COMPANY_DATA2/frontend'  # 처리할 폴더 경로

# Chroma 벡터 DB 초기화
db_dir = './front_chroma_db2'
vectorstore = Chroma(persist_directory=db_dir, embedding_function=embedding_model)

# 청킹을 위한 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# 폴더 내의 텍스트 파일 처리
def process_files_in_folder(folder_path: str, role: str):
    # 지정된 폴더 내의 모든 텍스트 파일 찾기
    txt_files = glob.glob(os.path.join(folder_path, "*.txt"))

    for txt_file in txt_files:
        print(f"Processing file: {txt_file}")
        with open(txt_file, 'r', encoding='utf-8') as f:
            text = f.read()

        # 문서 청킹
        chunks = text_splitter.split_text(text)

        # 청크를 Document 객체로 변환
        documents = []
        for chunk in chunks:
            doc = Document(
                page_content=chunk,
                metadata={
                    "sourcefile": os.path.basename(txt_file),  # 파일명
                    "role": role,  # 직급 권한 (폴더 이름)
                    "last_edit": "2025-08-19"  # 마지막 수정일 고정
                }
            )
            documents.append(doc)

        # 배치로 문서들 추가 (올바른 메서드 사용)
        if documents:
            vectorstore.add_documents(documents)
            print(f"Added {len(documents)} chunks from {os.path.basename(txt_file)}")

# 데이터 처리 시작
if __name__ == "__main__":
    try:
        # role은 폴더 이름으로 설정
        role = os.path.basename(folder_path)  # 폴더 이름을 role로 사용
        process_files_in_folder(folder_path, role)
        print("모든 문서가 성공적으로 벡터 데이터베이스에 저장되었습니다.")
    except Exception as e:
        print(f"오류 발생: {e}")


Processing file: ./COMPANY_DATA2/frontend\01_frontend__프론트엔드_아키텍처_문서.txt
Added 4 chunks from 01_frontend__프론트엔드_아키텍처_문서.txt
Processing file: ./COMPANY_DATA2/frontend\02_frontend__API_연동_매뉴얼.txt
Added 4 chunks from 02_frontend__API_연동_매뉴얼.txt
Processing file: ./COMPANY_DATA2/frontend\03_frontend__빌드_배포_가이드.txt
Added 3 chunks from 03_frontend__빌드_배포_가이드.txt
Processing file: ./COMPANY_DATA2/frontend\04_frontend__코딩_컨벤션_&_스타일_가이드.txt
Added 4 chunks from 04_frontend__코딩_컨벤션_&_스타일_가이드.txt
Processing file: ./COMPANY_DATA2/frontend\05_frontend__UI_UX_디자인_가이드라인.txt
Added 5 chunks from 05_frontend__UI_UX_디자인_가이드라인.txt
Processing file: ./COMPANY_DATA2/frontend\06_frontend__테스트_전략_및_QA_가이드.txt
Added 4 chunks from 06_frontend__테스트_전략_및_QA_가이드.txt
Processing file: ./COMPANY_DATA2/frontend\07_frontend__보안_가이드라인.txt
Added 4 chunks from 07_frontend__보안_가이드라인.txt
Processing file: ./COMPANY_DATA2/frontend\08_frontend__로그_모니터링_가이드.txt
Added 4 chunks from 08_frontend__로그_모니터링_가이드.txt
Processing file: ./COM

In [6]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# BGE-M3 모델 로드 (BAAI/bge-m3 사용)
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# Chroma 벡터 DB 로드
db_dir = './front_chroma_db2'
vectorstore = Chroma(persist_directory=db_dir, embedding_function=embedding_model)

# 검색할 쿼리
query = "코드노바의 캐시 만료 시간은 어떤 기준으로 설정해야 하나요?"

# 벡터 DB에서 유사한 문서 검색
results = vectorstore.similarity_search(query, k=3)  # k는 검색할 상위 결과의 수 (예: 3)

# 검색된 결과 출력
for i, result in enumerate(results):
    print(f"Result {i+1}:")
    print(f"Content: {result.page_content}")
    print(f"Metadata: {result.metadata}")
    print("=" * 50)


Result 1:
Content: <!-- 회사: 코드노바 | 대상: 사원(프론트엔드) | 작성일: 2025-08-29 -->
# 코딩 컨벤션 & 스타일 가이드

분류: frontend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29

---

## 1. 기본 원칙

- **일관성**: 모든 팀원이 동일한 스타일을 따르도록 하여 코드의 가독성을 높입니다.
- **명확성**: 코드가 하는 일을 명확하게 표현하여 다른 개발자가 쉽게 이해할 수 있도록 합니다.
- **재사용성**: 중복 코드를 피하고, 재사용 가능한 컴포넌트를 만들어 유지보수성을 높입니다.

## 2. 파일 및 폴더 구조

- **폴더 구조**: 프로젝트의 구조는 기능별로 나누어 관리합니다. 각 기능은 별도의 폴더를 가지며, 그 안에 관련된 파일들을 포함합니다.
- **파일 네이밍**: 파일 이름은 소문자와 하이픈(-)을 사용하여 작성합니다. 예: `user-profile.js`

## 3. 변수 및 함수
Metadata: {'sourcefile': '04_frontend__코딩_컨벤션_&_스타일_가이드.txt', 'role': 'frontend', 'last_edit': '2025-08-19'}
Result 2:
Content: <!-- 회사: 코드노바 | 대상: 사원(프론트엔드) | 작성일: 2025-08-29 -->
# 보안 가이드라인
분류: frontend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29

---

## 1. 목적
이 문서는 코드노바 프론트엔드팀의 보안 강화를 위한 지침을 제공합니다. 개발 과정에서 보안 취약점을 최소화하고, 사용자 데이터를 안전하게 보호하기 위해 필요한 절차를 명시합니다.

## 2. 일반 보안 원칙
- **최소 권한 원칙**: 각 개발자는 필요한 최소한의 권한만을 부여받아야 합니다.
- **정기적인 교육**: 팀원들은 보안 관련 교육을 정기적으로 이수해야 합니다.
- **보안 점검**: 코드 배포 전에는 반드시